# Задание
1. Реализуйте алгоритм **Dueling	DQN** (дуэльные	DQN) и с его помощью решите задачу MountainCar.
2. Создайте видео эпизода с наибольшим вознаграждением и вставьте его в колаб.

In [ ]:
#@title Установка зависимостей
!pip install --upgrade gymnasium numpy matplotlib torch > /dev/null 2>&1

In [ ]:
#@title Импорты
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import time
import os
import base64
from IPython import display as ipythondisplay

# Совместимость gym numpy
np.bool8 = np.bool_

In [ ]:
#@title Константы и гиперпараметры

ENV_NAME = "MountainCar-v0"
N_EPISODES = 1000
GAMMA = 0.99          # Коэффициент обесценивания будущих наград
EPSILON_START = 0.3   # Начальная вероятность случайного действия
EPSILON_END = 0.01    # Конечная вероятность случайного действия
EPSILON_DECAY = 0.99  # Скорость уменьшения epsilon
LR = 0.01             # Скорость обучения нейронной сети
HIDDEN_SIZE = 64      # Размер скрытых слоев нейронной сети

In [ ]:
#@title Создание среды

env = gym.make(ENV_NAME, render_mode='rgb_array')
n_state = env.observation_space.shape[0]
n_action = env.action_space.n

print(f"Размерность состояния: {n_state}")
print(f"Количество действий: {n_action}")

Размерность состояния: 2
Количество действий: 3


In [ ]:
#@title Архитектура Dueling DQN

class DuelingDQN(nn.Module):
    """
    Архитектура Dueling DQN, разделяющая Q-функцию на Value и Advantage потоки.

    Ключевая идея: Q(s,a) = V(s) + (A(s,a) - mean(A(s,a)))
    где V(s) - ценность состояния, A(s,a) - преимущество действия
    """
    def __init__(self, n_state, n_action, hidden_size=128):
        super(DuelingDQN, self).__init__()

        # Общий слой признаков для извлечения характеристик состояния
        self.feature_layer = nn.Sequential(
            nn.Linear(n_state, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )

        # Поток для оценки ценности состояния V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1)
        )

        # Поток для оценки преимущества действий A(s,a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, n_action)
        )

    def forward(self, state):
        # Извлекаем общие признаки состояния
        features = self.feature_layer(state)

        # Оцениваем ценность состояния
        value = self.value_stream(features)

        # Оцениваем преимущество каждого действия
        advantage = self.advantage_stream(features)

        # Комбинируем: вычитаем среднее для стабильности обучения
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

In [ ]:
#@title Класс агента Dueling DQN

class DuelingDQNAgent:
    """
    Агент Dueling DQN для обучения с подкреплением.

    Использует две нейронные сети:
    - q_network: основная сеть для оценки Q-значений
    - target_network: целевая сеть для стабильности обучения
    """
    def __init__(self, n_state, n_action, hidden_size=128, lr=0.001, gamma=0.99,
                 epsilon_start=0.3, epsilon_end=0.01, epsilon_decay=0.995):
        self.n_action = n_action
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay

        # Определяем устройство (GPU или CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Создаем основную и целевую сети
        self.q_network = DuelingDQN(n_state, n_action, hidden_size).to(self.device)
        self.target_network = DuelingDQN(n_state, n_action, hidden_size).to(self.device)
        # Инициализируем целевую сеть весами основной сети
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()  # Целевая сеть в режиме оценки

        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)

    def predict(self, state):
        """
        Вычисляет значения Q-функции для всех действий в данном состоянии.

        Args:
            state: Состояние среды

        Returns:
            Q-значения для всех действий
        """
        with torch.no_grad():  # Отключаем градиенты для инференса
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_network(state_tensor)
            return q_values

    def update(self, state, target_q_values):
        """
        Обновляет веса сети на основе состояния и целевых Q-значений.

        Используется прямое обновление на каждом шаге (без replay buffer).

        Args:
            state: Текущее состояние
            target_q_values: Целевые Q-значения для всех действий

        Returns:
            Значение функции потерь
        """
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        target_tensor = torch.FloatTensor(target_q_values).unsqueeze(0).to(self.device)

        # Вычисляем текущие Q-значения
        current_q_values = self.q_network(state_tensor)
        # Вычисляем потерю как MSE между текущими и целевыми Q-значениями
        loss = F.mse_loss(current_q_values, target_tensor)

        # Обновляем веса сети
        self.optimizer.zero_grad()
        loss.backward()
        # Обрезаем градиенты для стабильности обучения
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()

        return loss.item()

    def update_target_network(self):
        """Обновляет целевую сеть весами основной сети."""
        self.target_network.load_state_dict(self.q_network.state_dict())

In [ ]:
#@title Вспомогательные функции

def modify_reward(state, reward):
    """
    Модифицирует вознаграждение для ускорения обучения.
    В MountainCar стандартное вознаграждение всегда -1, что затрудняет обучение.
    Эта функция добавляет бонусы за продвижение к цели (позиция >= 0.5).

    Args:
        state: Состояние среды [позиция, скорость]
        reward: Исходное вознаграждение

    Returns:
        Модифицированное вознаграждение
    """
    position = state[0]
    # Базовое вознаграждение зависит от позиции
    modified = position + 0.5

    # Добавляем бонусы за достижение определенных позиций
    if position >= 0.5:     # Достигнута цель
        modified += 100
    elif position >= 0.25:  # Близко к цели
        modified += 20
    elif position >= 0.1:   # Прогресс в правильном направлении
        modified += 10
    elif position >= 0:     # Небольшой прогресс
        modified += 5

    return modified

def gen_epsilon_greedy_policy(agent, epsilon, n_action):
    """
    Генерирует ε-жадную политику для выбора действий.

    Args:
        agent: Агент DuelingDQNAgent
        epsilon: Вероятность случайного действия
        n_action: Количество возможных действий

    Returns:
        Функция политики, принимающая состояние и возвращающая действие
    """
    def policy_function(state):
        if random.random() < epsilon:
            # Случайное действие для исследования
            return random.randint(0, n_action - 1)
        else:
            # Жадное действие на основе Q-значений
            q_values = agent.predict(state)
            return int(q_values.argmax().item())
    return policy_function

def update_best_episode(current_reward, current_sequence, current_reached_goal,
                        best_reward, best_sequence, episode_num):
    """
    Определяет, нужно ли обновить лучший эпизод.
    Приоритет отдается эпизодам, достигшим цели.

    Args:
        current_reward: Вознаграждение текущего эпизода
        current_sequence: Последовательность действий текущего эпизода
        current_reached_goal: Флаг достижения цели в текущем эпизоде
        best_reward: Текущее лучшее вознаграждение
        best_sequence: Текущая лучшая последовательность действий
        episode_num: Номер текущего эпизода (для вывода сообщений)

    Returns:
        (should_update, new_best_reward, new_best_sequence):
        Флаг обновления, новое лучшее вознаграждение, новая лучшая последовательность
    """
    should_update = False
    new_best_reward = best_reward
    new_best_sequence = best_sequence

    if best_sequence is None:
        # Первый эпизод
        should_update = True
        new_best_reward = current_reward
        new_best_sequence = current_sequence.copy()
    elif current_reached_goal:
        # Текущий эпизод достиг цели
        previous_reached_goal = (best_reward > -150)
        if not previous_reached_goal or current_reward > best_reward:
            should_update = True
            new_best_reward = current_reward
            new_best_sequence = current_sequence.copy()
    elif current_reward > best_reward:
        # Текущий эпизод лучше, но не достиг цели
        previous_reached_goal = (best_reward > -150)
        if not previous_reached_goal:
            should_update = True
            new_best_reward = current_reward
            new_best_sequence = current_sequence.copy()

    return should_update, new_best_reward, new_best_sequence

In [ ]:
#@title Функция обучения

def train_agent(env, agent, n_episodes, gamma=0.99, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.99):
    """
    Обучение агента Dueling DQN с прямым обновлением на каждом шаге.

    Args:
        env: Среда Gymnasium
        agent: Агент DuelingDQNAgent
        n_episodes: Количество эпизодов обучения
        gamma: Коэффициент обесценивания
        epsilon_start: Начальное значение epsilon
        epsilon_end: Конечное значение epsilon
        epsilon_decay: Коэффициент затухания epsilon

    Returns:
        episode_rewards: Список вознаграждений по эпизодам
        episode_steps: Список количества шагов по эпизодам
        best_reward: Лучшее вознаграждение
        best_episode_sequence: Последовательность действий лучшего эпизода
    """
    episode_rewards = []
    episode_steps = []
    best_reward = float('-inf')
    best_episode_sequence = None  # Сохраняем последовательность лучшего эпизода
    epsilon = epsilon_start

    for episode in range(n_episodes):
        policy = gen_epsilon_greedy_policy(agent, epsilon, agent.n_action)
        state, info = env.reset()
        total_reward = 0
        steps = 0
        episode_sequence = []  # Сохраняем последовательность действий текущего эпизода
        episode_reached_goal = False  # Флаг достижения цели

        while True:
            action = policy(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            steps += 1

            # Сохраняем действие для последующего воспроизведения
            episode_sequence.append(int(action))

            # Проверяем, достигнута ли цель (позиция >= 0.5)
            if next_state[0] >= 0.5:
                episode_reached_goal = True

            # Модифицируем вознаграждение для ускорения обучения
            modified_reward = modify_reward(next_state, reward)

            # Получаем текущие Q-значения для всех действий
            q_values = agent.predict(state).squeeze().cpu().numpy().tolist()

            if done or truncated:
                # Для терминального состояния: Q(s,a) = r
                q_values[action] = modified_reward
                agent.update(state, q_values)
                break
            else:
                # Для нетерминального состояния: Q(s,a) = r + γ * max Q(s',a')
                q_values_next = agent.target_network(
                    torch.FloatTensor(next_state).unsqueeze(0).to(agent.device)
                )
                q_values[action] = modified_reward + gamma * q_values_next.max().item()
                agent.update(state, q_values)

            state = next_state

        # Уменьшаем epsilon
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        agent.epsilon = epsilon

        # Периодически обновляем целевую сеть для стабильности обучения
        if episode % 10 == 0:
            agent.update_target_network()

        episode_rewards.append(total_reward)
        episode_steps.append(steps)

        # Сохраняем лучший эпизод (приоритет эпизодам, достигшим цели)
        should_update, best_reward, best_episode_sequence = update_best_episode(
            current_reward=total_reward,
            current_sequence=episode_sequence,
            current_reached_goal=episode_reached_goal,
            best_reward=best_reward,
            best_sequence=best_episode_sequence,
            episode_num=episode
        )

        if episode % max(1, n_episodes // 10) == 0:
            avg_reward = np.mean(episode_rewards[-100:]) if len(episode_rewards) >= 100 else np.mean(episode_rewards)
            print(f"Эпизод {episode}/{n_episodes} | Средний результат={avg_reward:.2f} | "
                  f"Лучший результат={best_reward:.2f} | Epsilon={epsilon:.3f}")

    return episode_rewards, episode_steps, best_reward, best_episode_sequence

In [ ]:
#@title Обучение агента

agent = DuelingDQNAgent(
    n_state=n_state,
    n_action=n_action,
    hidden_size=HIDDEN_SIZE,
    lr=LR,
    gamma=GAMMA,
    epsilon_start=EPSILON_START,
    epsilon_end=EPSILON_END,
    epsilon_decay=EPSILON_DECAY
)

print("Начало обучения...")
start_time = time.time()
episode_rewards, episode_steps, best_reward, best_episode_sequence = train_agent(
    env=env,
    agent=agent,
    n_episodes=N_EPISODES,
    gamma=GAMMA,
    epsilon_start=EPSILON_START,
    epsilon_end=EPSILON_END,
    epsilon_decay=EPSILON_DECAY
)

elapsed_time = time.time() - start_time
print(f"\nОбучение завершено за {elapsed_time:.1f} сек.")

Начало обучения...
Эпизод 0/1000 | Средний результат=-200.00 | Лучший результат=-200.00 | Epsilon=0.297
Эпизод 100/1000 | Средний результат=-199.73 | Лучший результат=-173.00 | Epsilon=0.109
Эпизод 200/1000 | Средний результат=-200.00 | Лучший результат=-173.00 | Epsilon=0.040
Эпизод 300/1000 | Средний результат=-199.72 | Лучший результат=-178.00 | Epsilon=0.015
Эпизод 400/1000 | Средний результат=-198.86 | Лучший результат=-86.00 | Epsilon=0.010
Эпизод 500/1000 | Средний результат=-198.91 | Лучший результат=-86.00 | Epsilon=0.010
Эпизод 600/1000 | Средний результат=-198.91 | Лучший результат=-86.00 | Epsilon=0.010
Эпизод 700/1000 | Средний результат=-198.55 | Лучший результат=-86.00 | Epsilon=0.010
Эпизод 800/1000 | Средний результат=-200.00 | Лучший результат=-86.00 | Epsilon=0.010
Эпизод 900/1000 | Средний результат=-200.00 | Лучший результат=-86.00 | Epsilon=0.010

Обучение завершено за 893.9 сек.


In [ ]:
#@title Функции для записи и отображения видео

def show_video(video_path):
    """Отображение видео в Google Colab."""
    video = open(video_path, "rb").read()
    video_encoded = base64.b64encode(video).decode('utf-8')
    video_tag = '<video controls alt="test" src="data:video/mp4;base64,{0}">'.format(video_encoded)
    ipythondisplay.display(ipythondisplay.HTML(data=video_tag))

def record_best_episode_from_training(episode_sequence, video_dir="./video"):
    """
    Записывает видео лучшего эпизода из обучения.
    Воспроизводит сохраненную последовательность действий для создания видео.

    Args:
        episode_sequence: Список действий лучшего эпизода
        video_dir: Директория для сохранения видео

    Returns:
        Путь к созданному видеофайлу или None при ошибке
    """
    if episode_sequence is None or len(episode_sequence) == 0:
        print("Последовательность лучшего эпизода не найдена.")
        return None

    # Поддерживаем разные форматы
    actions = []
    for item in episode_sequence:
        if isinstance(item, tuple):
            # Если это кортеж, извлекаем действие
            if len(item) == 2:
                # Определяем, какой элемент - действие (int от 0 до 2)
                if isinstance(item[0], (int, np.integer)) and 0 <= item[0] <= 2:
                    actions.append(int(item[0]))
                elif isinstance(item[1], (int, np.integer)) and 0 <= item[1] <= 2:
                    actions.append(int(item[1]))
                elif hasattr(item[0], 'shape') or isinstance(item[0], (list, np.ndarray)):
                    # Первый элемент - state (массив), второй - action
                    actions.append(int(item[1]))
                elif hasattr(item[1], 'shape') or isinstance(item[1], (list, np.ndarray)):
                    # Второй элемент - state (массив), первый - action
                    actions.append(int(item[0]))
        elif isinstance(item, (int, np.integer, float)):
            # Уже действие, проверяем валидность
            action_int = int(item)
            if 0 <= action_int <= 2:
                actions.append(action_int)

    if len(actions) == 0:
        print("Не удалось извлечь действия из последовательности.")
        return None

    os.makedirs(video_dir, exist_ok=True)

    # Создаем среду с записью видео
    episode_env = gym.wrappers.RecordVideo(
        gym.make(ENV_NAME, render_mode='rgb_array'),
        video_dir,
        episode_trigger=lambda x: x == 0,  # Записываем только первый эпизод
        name_prefix="best_episode"
    )

    # Фиксируем начальное состояние для воспроизводимости видео
    state, info = episode_env.reset(seed=41)

    # Воспроизводим сохраненную последовательность действий
    for action in actions:
        state, reward, terminated, truncated, _ = episode_env.step(action)

        if terminated or truncated:
            break

    episode_env.close()

    # Находим созданный видеофайл
    video_files = [f for f in os.listdir(video_dir)
                  if f.endswith(".mp4") and "best_episode" in f]
    if video_files:
        latest_file = max(video_files, key=lambda f: os.path.getmtime(os.path.join(video_dir, f)))
        video_path = os.path.join(video_dir, latest_file)
        return video_path

    return None

In [ ]:
#@title Запись видео эпизода с наибольшим вознаграждением

print("Запись видео эпизода с наибольшим вознаграждением из обучения...")
print(f"Вознаграждение лучшего эпизода: {best_reward:.2f}\n")

video_path = record_best_episode_from_training(best_episode_sequence)

if video_path:
    print(f"Видео успешно записано: {video_path}")
    print("\nОтображение видео:")
    show_video(video_path)
else:
    print("Не удалось записать видео.")

Запись видео эпизода с наибольшим вознаграждением из обучения...
Вознаграждение лучшего эпизода: -86.00

Видео успешно записано: ./video/best_episode-episode-0.mp4

Отображение видео:


# Выводы по проделанной работе

В рамках данной работы мной была реализована система обучения с подкреплением для среды MountainCar с использованием алгоритма Dueling DQN (дуэльные глубокие Q-сети). Я использовал библиотеку PyTorch для создания нейронных сетей и Gymnasium для работы со средой. Архитектура Dueling DQN разделяет оценку Q-функции на два потока: поток оценки ценности состояния V(s) и поток оценки преимущества действий A(s,a), что позволяет сети лучше оценивать ценность состояний независимо от конкретных действий.

Для улучшения качества обучения была реализована модификация вознаграждений, которая добавляет бонусы за продвижение к цели, так как в MountainCar стандартное вознаграждение всегда равно -1, что затрудняет обучение. Я использовал ε-жадную стратегию для баланса между исследованием и эксплуатацией, а также реализовал целевую сеть (target network), которая обновляется периодически для стабилизации процесса обучения. Мной был применен метод градиентного клиппинга для предотвращения взрывающихся градиентов.

В соответствии с заданием, я реализовал функцию записи и отображения видео лучшего эпизода из обучения, что позволяет визуально оценить работу обученного агента. В процессе обучения агент достиг вознаграждения -86.00, что демонстрирует возможность применения алгоритма Dueling DQN для решения задачи MountainCar.